In [1]:
import psycopg2
import requests
import json

In [2]:
# Parámetros de conexión a la base de datos
db_host = 'localhost'
db_name = 'cookwe'
db_user = 'postgres'
db_password = 'admin'
db_port = '5434'  # Puerto por defecto de PostgreSQL

# Establecer la conexión
conn = psycopg2.connect(host=db_host, port=db_port, database=db_name, user=db_user, password=db_password)
cursor = conn.cursor()

In [3]:
# Verificar la conexión a la base de datos
if conn is not None and conn.closed == 0:
    print("Conexión exitosa a la base de datos")
else:
    print("Error al conectar a la base de datos")


Conexión exitosa a la base de datos


In [12]:
# Obtener las preferencias del usuario
def obtener_preferencias_usuario(usuario_id):
    cursor.execute("SELECT idrecipe FROM recipe_user WHERE iduser = %s", (usuario_id,))
    return cursor.fetchall()

def obtener_recomendaciones(usuario_id):
    cursor.execute("SELECT DISTINCT idrecipe FROM recipe_user WHERE iduser != %s", (usuario_id,))
    recetas_favoritas_otros_usuarios = cursor.fetchall()

    recomendaciones = []
    for receta_favorita in recetas_favoritas_otros_usuarios:
        receta_id = receta_favorita[0]
        
        if receta_id not in [r[0] for r in recomendaciones]:
            recomendaciones.append((receta_id, None))  # Puedes dejar el valor de valoración como None

    # Verificar si hay recomendaciones con valoración None
    if any(r[1] is None for r in recomendaciones):
        recomendaciones = [(r[0], 0) if r[1] is None else r for r in recomendaciones]

    # Ordenar las recomendaciones según algún criterio
    recomendaciones.sort(key=lambda x: x[1], reverse=True)
    return recomendaciones[:10]  # Obtener las 10 mejores recomendaciones



In [14]:
usuario_id = 24
preferencias = obtener_preferencias_usuario(usuario_id)
recomendaciones = obtener_recomendaciones(usuario_id)
print(preferencias)
print(recomendaciones)

[]
[(52958, 0), (52903, 0), (52950, 0), (53021, 0), (52954, 0), (52945, 0), (52956, 0), (52872, 0), (53024, 0), (52949, 0)]


In [19]:
import requests

# Realizar solicitud GET a 'http://localhost:3000/obtener' para obtener el usuario_id
response = requests.get('http://localhost:3000/obtener')
print(response.json())  # Obtener el ID del usuario de la respuesta JSON

{'error': 'No autenticado'}


In [20]:
# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Hacer algo con el usuario_id obtenido
    preferencias = obtener_preferencias_usuario(usuario_id)
    recomendaciones = obtener_recomendaciones(usuario_id)
    print(preferencias)
    print(recomendaciones)

    # Realizar solicitud POST a 'http://localhost:3000/data' con las recomendaciones
    response = requests.post('http://localhost:3000/data', json=recomendaciones)
    if response.status_code == 200:
        print('Solicitud POST exitosa')
    else:
        print('Error en la solicitud POST:', response.status_code)
else:
    print('Error en la solicitud GET:', response.status_code)

Error en la solicitud GET: 401


In [21]:
from flask import Flask, request, jsonify
import psycopg2
from flask_cors import CORS

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "http://localhost:3000"}})

# Parámetros de conexión a la base de datos
db_host = 'localhost'
db_name = 'cookwe'
db_user = 'postgres'
db_password = 'admin'
db_port = '5434'

# Establecer la conexión a la base de datos
conn = psycopg2.connect(host=db_host, port=db_port, database=db_name, user=db_user, password=db_password)
cursor = conn.cursor()

# Obtener las preferencias del usuario
def obtener_preferencias_usuario(usuario_id):
    cursor.execute("SELECT idrecipe FROM recipe_user WHERE iduser = %s", (usuario_id,))
    return cursor.fetchall()

# Obtener las recomendaciones para un usuario
def obtener_recomendaciones_usuario(usuario_id):
    cursor.execute("SELECT DISTINCT idrecipe FROM recipe_user WHERE iduser != %s", (usuario_id,))
    recetas_favoritas_otros_usuarios = cursor.fetchall()

    recomendaciones = []
    for receta_favorita in recetas_favoritas_otros_usuarios:
        receta_id = receta_favorita[0]
        
        if receta_id not in [r[0] for r in recomendaciones]:
            recomendaciones.append((receta_id, None))  # Puedes dejar el valor de valoración como None

    # Verificar si hay recomendaciones con valoración None
    if any(r[1] is None for r in recomendaciones):
        recomendaciones = [(r[0], 0) if r[1] is None else r for r in recomendaciones]

    # Ordenar las recomendaciones según algún criterio
    recomendaciones.sort(key=lambda x: x[1], reverse=True)
    return recomendaciones[:10]  # Obtener las 10 mejores recomendaciones

@app.route('/recommendations', methods=['POST'])
def obtener_recomendaciones():
    usuario_id = request.json['usuario_id']

    preferencias = obtener_preferencias_usuario(usuario_id)
    recomendaciones = obtener_recomendaciones_usuario(usuario_id)
    print(preferencias)
    print(recomendaciones)

    return jsonify(recomendaciones)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [ ]:
pip install -U flask-cors

^C


In [ ]:
pip show flask-cors

In [1]:
from flask import Flask, request

app = Flask(__name__)

@app.route('/recibir', methods=['POST'])
def recibir_datos():
    usuario_id = request.json['usuario_id']
    
    # Haz algo con el ID de usuario recibido
    print('ID de usuario recibido:', usuario_id)
    
    return 'Datos recibidos en Jupyter Notebook'

if __name__ == '__main__':
    app.run(port=8888)

 * Serving Flask app '__main__'
 * Debug mode: off


Intento de acceso a un socket no permitido por sus permisos de acceso
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\werkzeug\serving.py", line 911, in prepare_socket
    s.bind(server_address)
OSError: [WinError 10013] Intento de acceso a un socket no permitido por sus permisos de acceso

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11532\1244911020.py", line 15, in <module>
    app.run(port=8888)
  File "c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\flask\app.py", line 1191, in run
    run_simple(t.cast(str, host), port, self, **options)
  File "c:\users\lenovo\appdata\local\programs\python\python37\lib\site-packages\werkzeug\serving.py", line 1062, in run_si

TypeError: object of type 'NoneType' has no len()